In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import pandas as pd
import numpy as np
import sqlite3
# DB 불러오기
connection = sqlite3.connect("/Users/kimtaewan/Desktop/SIG 개인폴더/MV_RECOMMEND/data.db")

# 테이블 읽기
data = pd.read_sql("SELECT * FROM data;", connection)

data
    
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(data.storyWord)

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

tfidf_matrix

<18097x45692 sparse matrix of type '<class 'numpy.float64'>'
	with 925686 stored elements in Compressed Sparse Row format>

In [3]:
"""인덱스 테이블 만들기"""
indices = pd.Series(data.index, index = data.title).drop_duplicates()
indices

title
트랜스포머: 비스트의 서막        0
신해혁명                  1
귀공자                   2
롱 듀얼: 최후의 결투          3
안티 레이싱                4
                  ...  
거미줄               18092
반지제왕:유령무사         18093
파이어하트             18094
에어 포스: 라스트 워      18095
쿠툴룬: 전쟁의 여신       18096
Length: 18097, dtype: int64

In [4]:
"""추천 해주기"""
def movie_REC(title, cosine_sim=cosine_sim):
    
    #입력한 영화로 부터 인덱스 가져오기
    idx = indices[title]

    # 모든 영화에 대해서 해당 영화와의 유사도를 구하기
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 영화들을 정렬
    sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse = True)

    # 가장 유사한 10개의 영화를 받아옴
    sim_scores = sim_scores[1:6]

    # 가장 유사한 10개 영화의 인덱스 받아옴
    movie_indices = [i[0] for i in sim_scores]
    
    #기존에 읽어들인 데이터에서 해당 인덱스의 값들을 가져온다. 그리고 스코어 열을 추가하여 코사인 유사도도 확인할 수 있게 한다.
    result_df = data.iloc[movie_indices].copy()
    result_df['score'] = [i[1] for i in sim_scores]
    
    # 읽어들인 데이터에서 출력 필요없는 부분 제거 
    del result_df['story']
    del result_df['storyWord']
    del result_df['poster']
    del result_df['actor']
    del result_df['directer']
    del result_df['time']
    del result_df['limit_age']
    del result_df['id']
    del result_df['theme']

    # 가장 유사한 10개의 영화의 제목을 리턴
    return result_df

In [5]:
movie_REC("범죄도시3")

,title,score,keyword1,keyword2,keyword3,keyword4,keyword5
1026,범죄도시2,0.270066,베트남,악행,한국,범죄,무자비하다
832,범죄도시,0.179676,리더,대한민국,서울,장악,이끌다
16132,프론트,0.171054,살인,강간,형사,도둑,사건
2000,적인걸9 : 구유업화,0.166417,밝히다,사건,죽음,일말,화재
3014,쉐퍼드 - 보더 패트롤,0.152537,경찰관,경찰,잔인하다,마약,텍사스
